#COMPSCI 546: Applied Information Retrieval
##Assignment 7: Learning to Rank & Neural Ranking Model (Total : 100 points)



# Download input files

Please execute the cell below to download the input files. 

In [ ]:

import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


import os
import zipfile
import numpy as np

download = drive.CreateFile({'id': '11K8r5a_Aj9S4Cpd8k3x76ccBTluNguip'})
download.GetContentFile('HW07.zip')

with zipfile.ZipFile('HW07.zip', 'r') as zip_file:
    zip_file.extractall('./')
os.remove('HW07.zip')
# We will use hw05 as our working directory
os.chdir('HW07')

#Setting the input files
passage_file = "antique-collection.tok.clean_kstem"
test_queries_file = "antique-test-queries.tok.clean_kstem"
train_queries_file = "antique-train-queries.tok.clean_kstem"
val_queries_file = "antique-val-queries.tok.clean_kstem"
sample = "sample.txt"
stopwords_file = "stopword_INQUERY"
val_baseline_features_file = "val_baseline_features_top10"
test_baseline_features_file = "test_baseline_features_top10"
train_baseline_features_file = "train_baseline_features_top10"


# 1 : Initial Data Setup (20 points)

We use collection from the ANTIQUE  [https://arxiv.org/pdf/1905.08957.pdf] dataset for this assignment. As described in the previous assignments, this is a passage retrieval dataset. 

The description of the input files provided for this assignment is given below.

**Collection file**

Each row of the file consists of the following information:

*passage_id  passage_text*

The id and text information is tab separated. The passage text has been pre-processed to remove punctutation, tokenised and stemmed using the Krovetz stemmer. The terms in the passage text can be accessed by splitting the text based on space.

**Query files**

You are provided with train,validation and test query files.  Each row of the file consists of the following information:

*query_id  query_text*
 
The id and text information is tab separated. The query text has been pre-processed to remove punctutation, tokenised and stemmed using the Krovetz stemmer. The terms in the text can be accessed by splitting the text based on space.

**Feature files**

You are provided with train,validation and test feature files. Each row of the file consists of the following information:

*query_id  passage_id relevance_score vsm_score bm25_score*

Each row contains features for a (query,passage) pair and is space separated. The relevance_score is the human annotated relevance score. vsm_score and bm25 scores are the relevance scores for the pair corresponding to the two different scoring methods. 

**Stopwords file**

The stopword file contains the list of stopwords. This file has a stopword per line.

**Sample file**

For this assignment, we use the pyltr Learning to Rank framework from [https://github.com/harshhpareek/pyltr]. The input file to the framework has to be set similar to the sample.txt in the following format:

*relevance_score qid:query_id 1:feature1 2:feature2 #docid = passage_id*

Each entry is space separated. This file has been provided only for reference to create files of the same format.

In the cell below, you have to implement the following:


*   Load the query files
*   Load the collection
*   Load the stopwords




In [ ]:
from collections import defaultdict
''' 
In this function, load the query files into dict 
Return Variables: 
queries - dict with qid as key and querytext as value
'''
def loadQueryFile(filename):
    #enter your code here 
    queries = defaultdict()
    file = open(filename, "r")
    for q in file:
      temp = q.split()
      queries[temp[0]] = temp[1:]
    file.close()
    return queries


''' 
In this function, load the collection into dict 
Return Variables: 
coll - dict with passage id as key and passage text as value
'''
def loadCollection(passage_file):
    #enter your code here 
    coll = defaultdict()
    file = open(passage_file, "r")
    for f in file:
      temp = f.split()
      coll[temp[0]] = temp[1:]
    file.close()
    return coll

''' 
In this function, load the stopwords into dict 
Return Variables: 
stopwords - dict with stopword as key 
'''
def loadStopWords(stopwords_file):
    #enter your code here 
    stopwords = defaultdict()
    file = open(stopwords_file, "r")
    for s in file:
      temp = s.split()
      stopwords[temp[0]] = temp[1:]
    file.close()   
    return stopwords


train_queries = loadQueryFile(train_queries_file)
val_queries = loadQueryFile(val_queries_file)
test_queries = loadQueryFile(test_queries_file)
coll = loadCollection(passage_file)
stopwords = loadStopWords(stopwords_file)

print('Total Number of train queries: {0}'.format(len(train_queries)))
print('Total Number of validation queries: {0}'.format(len(val_queries)))
print('Total Number of test queries: {0}'.format(len(test_queries)))
print('Total Number of passages in the collection: {0}'.format(len(coll)))
print('Total Number of stopwords: {0}'.format(len(stopwords)))


Total Number of train queries: 2226
Total Number of validation queries: 200
Total Number of test queries: 200
Total Number of passages in the collection: 403492
Total Number of stopwords: 418


# 2 : Feature Preparation (20 points)

The input feature file consists of two main features : VSM score and bm25 score of query,passage pairs. In this section, you will implement three additional features and use the information to create input feature file which contains the 5 features. The feature file must have the same format as sample. 

In the cell below, implement the following features:

*  Number of unique term overlap between query and passage after excluding stopwords and words with only one character from both.

  [ Example = Query : why do a cat headbutt

  Passage : cat fight for attention and domination if you show a can of food to my cat he headbutt it. 

  Number of Overlapped terms for the query/passage pair: 2  ]

*  Number of terms in query
*  Number of terms in passage


In [ ]:
''' 
In this function, create new feature file with additional features in the format required as input   
Return Variables: 
There is no return variable. You would create a new feature file "final_features_file"
One example of the row of the newly created file is 
"0 qid:3990512 1:3.5053628162466897 2:10.841493137122296 3:1 4:6 5:112 #docid = 882429_10"
The format of the file is: 
"relevance_score qid:query_id 1:feature1 2:feature2 3:feature3 4:feature4 5:feature5 #docid = passage_id"
You can read through the input baseline_features_file, create additional features and 
add the updated entry into the new file.  
'''

def featureCreation(baseline_features_file, stopwords, train_queries, coll, final_features_file):
    #enter your code here
    file1 = open(baseline_features_file, 'r')
    #file2 = open(final_features_file, 'w')

    with open(final_features_file,"w") as file2:
    
      for line in file1:
        temp = line.rstrip().split(' ')
        rel_score = temp[2]
        qid = temp[0]
        pid = temp[1]
        f1 = temp[3]
        f2 = temp[4]

        #removing stopwords from query and passages
        filtered_query = [word for word in train_queries[qid] if word not in stopwords]
        filtered_passg = [word for word in coll[pid] if word not in stopwords]

        f3 = str(len(set(filtered_query).intersection(set(filtered_passg)))) #uniq terms common to query and passage after removing stopwords
        f4 = str(len(train_queries[qid])) #num of terms in query
        f5 = str(len(coll[pid])) #num of terms in passage

        final_line = rel_score + " " + 'qid:' + qid + ' ' + '1:' + f1 + ' ' + '2:' + f2 + ' ' + '3:' + f3 + ' ' + '4:' + f4 + ' ' + '5:' + f5 + ' ' + '#docid' + ' ' + '=' + ' ' + pid
        file2.write(final_line+'\n')

    file1.close()

featureCreation(train_baseline_features_file, stopwords, train_queries, coll, 'train_features_final') 
featureCreation(val_baseline_features_file, stopwords, val_queries, coll, 'val_features_final') 
featureCreation(test_baseline_features_file, stopwords, test_queries, coll, 'test_features_final') 


# 3 : Model Training and Evaluation (20 points)

In the cell below, the pyltr library is used to train and evaluate ANTIQUE data using LambdaMART model. This takes less than a minute to execute. 


In [ ]:
#!pip install pyltr
#!pip install --upgrade scikit-learn==0.20.3

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pyltr
with open('train_features_final') as trainfile, open('val_features_final') as valifile, open('test_features_final') as evalfile:   
    TX, Ty, Tqids, _ = pyltr.data.letor.read_dataset(trainfile)
    VX, Vy, Vqids, _ = pyltr.data.letor.read_dataset(valifile)
    EX, Ey, Eqids, _ = pyltr.data.letor.read_dataset(evalfile)


metric = pyltr.metrics.NDCG(k=10)
monitor = pyltr.models.monitors.ValidationMonitor(VX, Vy, Vqids, metric=metric, stop_after=250)

model = pyltr.models.LambdaMART(
    metric=metric,
    n_estimators=100,
    learning_rate=0.02,
    max_features=0.5,
    query_subsample=0.5,
    max_leaf_nodes=10,
    min_samples_leaf=64,
    verbose=1,
)
model.fit(TX, Ty, Tqids,monitor=monitor)
Epred = model.predict(EX)
print ('LambdaMART model test score :'+ str(metric.calc_mean(Eqids, Ey, Epred)))
    

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/gradient_boosting.py:34: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from ._gradient_boosting import predict_stages
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/gradient_boosting.py:34: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from ._gradient_boosting import predict_stages


Early termination at iteration  99
LambdaMART model test score :0.800108667460195


### 3.1 : Describe how LambdaMART works. A brief description of the model and training objective would be sufficient. (10 points)

ANSWER:  
LambdaMart is a learning to rank algorithm which is a combination of LambdaRank and MART. The LambdaMart algorithm considers a pair of items (query, documents) at any given time and then comes up with a suitable ordering of those items before initiating the final order of the entire list. MART stands for Multiple Additive Regression Tress that is basically many decision trees added together and is combined with an ML technique called Gradient Boosting where a model is formed that is a weighted combination of an ensemble of “weak learners”. Here, each “weak learner” is a decision tree.

Thus, MART is a framework for finding an ensemble of trees that, when added together, minimizes a loss function. But, to this framework, gradients of the loss function must be fed at each training point. This is where the “Lambda” part of LambdaMART comes into the picture.

In ranking, optimising the loss function such as NDCG or MAP is key. But, these loss functions aren’t differentiable at all points, so they can’t used directly in the Gradient Boosting. Hence, a lambda gradient is derived which is analogous to those that have been defined for NDCG or MAP.

LambdaMART uses the idea from a previous model called LambdaRank. For every training point pair a lambda gradient is calculated. This gradient can be thought of as the factor that moves the documents up and down a ranked list. These lambda gradients are used for optimising loss functions such as NDCG, MAP and MRR.

Thus, In LambdaMART, gradient boosting framework with the pairwise lambda values acting as gradients is used, which leads to the leaf values of the tree getting updated during training.





### 3.2 : Can we directly optimize **typical** ranking metrics, such as NDCG or MAP? If yes, how? If no, why? (10 points)

ANSWER:  
Listwise approach aims to optimise evaluation metrics like NDCG and MAP. In this, each ranking list of documents for a query is treated as training instance. NDCG and MAP can’t be optimised using gradient descent-based methods as they are not differentiable. However, there are many ways in which these metrics taken as loss functions can be made differentiable. These include -
1. The approach as seen in LambdaRank and LambdaMART, where the ranking metrics are used to re-weight instances iteratively.
2. The approach as seen in SoftRank, where by smoothening the probabilistic scores, the probability distributions for the ranks of each document are calculated. Then, SoftNDCG which is the expected NDCG over the rank distribution is optimised.
3. The approach as seen in ListNet, where each ranked list corresponds to a permutation and a loss is defined over space of permutations. Here, loss is computed as the Binary Cross-Entropy distance between true and predicted probability distributions over the space of permutations.
4. The approach of LambdaLoss, which has created a framework wherein it allows to define metric-driven loss function related to the ranking metrics that need to be optimised. It defines the loss function as the negative log likelihood of a mixture model of the ranked lists.
Thus, we can say that these ranking metrics like NDCG and MAP can be optimised.

# 4 : Re-Ranking with BERT (40 points)
Assume that instead of manual feature engineering and using LambdaMART, you are asked to design a re-ranking model using BERT. Unfortunately, successful training of most neural ranking models for ranking would take multiple hours. Therefore, this part of the assignment does not contain coding questions. You are highly encouraged to implement a BERT Re-Ranking model for the ANTIQUE dataset, if you are interested in learning more about neural ranking models. HuggingFace provides nice tools for implementing Transformer-based models. 

Answer the following questions. 



### 4.1 : Describe your model architecture with all the details, including the model's input and the network architecture. You can describe the architecture in words, or insert an image containing the model architecture. (15 points)

ANSWER:  
In the pipeline, a large number of relevant documents for a given query are retrieved from a collection by using a probabilistic model like BM25. Then, passage re-ranking is done, wherein each document is scored against the query and it is re-ranked. From this final ranking list, the top k-documents are given as output of the pipeline.

The re-ranking model is created with BERT. This model will estimate a score $s_i$ for each passage $d_i$ against the query $q$. An input vector with a maximum length of 512 tokens is fed to the BERT. Query is fed as sentence A and passage text is fed as sentence B. Out of the 512 tokens, the query is truncated such that it can have a maximum of 64 tokens. The remaining 448 tokens will consist of the separator token and a concatenated passage. A [SEP] token is used to separate sentence A and sentence B and [CLS] token is used at the start of the input vector. CLS stands for classification and is used to represent sentence-level classification in BERT. The input vector consists of [CLS] + sentence A + [SEP] + sentence B + [SEP].

The BERT model is used as a binary classification model wherein we can feed the input vector to a single layer neural network to get an output probability of the passage being relevant. Probability for each passage is calculated against the given query and final list of passages can be obtained by ranking them from the highest probability to the lowest. The training of the BERT model can be started by using a pre-trained BERT model for which hyperparameter tuning can be done by using a suitable loss function.
 

### 4.2 : Write down a pointwise and a pairwise loss function of your choice for training the model. Which one do you expect perform better and why? (15 points)

ANSWER:  
For the above model, a pointwise loss function can be defined using the cross-entropy loss for training the BERT re-ranking model. Here, the loss function is defined for a query-document pair. The cross entropy loss is - 

$L = - {\displaystyle \sum_{j\in J_{\text{pos}}}} log(s_j) - {\displaystyle \sum_{j\in J_{\text{neg}}}} log(1 - s_j)$

wherein, $J_{\text{pos}}$ is the set of indexed of the relevant passages and $J_{\text{neg}}$ is the set of indexes of the non-relevant passages in the top-n documents retrieved with BM25.

A pairwise loss function can also be defined using pairwise hinge loss for training the BERT re-ranking model. Here, the goal of pairwise ranking is to rank documents correctly rather than predicting accurate relevance scores for each document. The loss function is defined for a query and two documents. The assumption made here is that the ground truth relevance label of the passages for each query is known.  The hinge loss is - 

$L(q,p_i,p_j) = max\{0, \epsilon - sign(Pos_{p_i} - Pos_{p_j})(S(q,p_i) - S(q,p_j)\}$

wherein, Pos(p) is the ground truth ranking position of the passage p, S is the score from the BERT model and ϵ
is the hyperparameter which determines the margin of the hinge loss. 

Between the two approaches, the pairwise approach will perform better and is more effective than pointwise approach. 

In pointwise loss function, we are optimising the model for being close to label and not for ranking documents. Thus, the order of the documents is compromised. Whereas, in pairwise approach, predicting relative order is closer to the nature of ranking than predicting class label or relevance score (pointwise approach). 

Loss in pairwise approach is based on pairs of documents with difference in relevance scores. The hinge loss pushes the score of the relevant document to be greater than the less relevant document. The scores don't need to match the labels, rather they should be properly ranked. 

Given the problem of ranking of the relevant documents, the pairwise hinge loss function is a better approach.

### 4.3 : If you want to use your model for a retrieval setting (instead of re-ranking a small number of documents), how do you design the model? Just a high-level description (a few sentences) would be sufficient. (10 points)

ANSWER:  
An efficient retrieval model is the one which makes use of an inverted index. Inverted index can make retrieval fast because of the natural language property of sparsity. Words are sparse in documents, i.e. not all words in vocabulary will be present in all the documents. This helps in keeping the posting list short and easy to use. 

A model which can learn these sparse representations of a document and make use of them for efficient retrieval is needed. The model should be trained on maximising the sparsity ratio i.e. minimising the $L_0$ loss of the vector, by minimising the number of number of non zero dimensions. Sparsity is defined by the number of non zero words in the vector.  

The model requires two loss functions - 
1. Pairwise hinge loss for promoting relevance 
2. Sparsity Loss to promote sparsity i.e. to force the representations to be extremely sparse. 

The model consists of an N-gram model with an hour-glass neural network. A sequence of words is fed into the model, wherein it is first converted into a sequence of word vectors. Then, it is fed into the hour glass neural network, wherein the hourglass network reduces the dimensionality to learn abstract and semantic representations and increase dimensionality for getting super high dimensional space. Then the higher dimensional vectors are fed into the average pooling layer where they are summed over a number of iterations and they give out a latent sparse representation/dimension.

An inverted index is then constructed with these sparse representations wherein the latent dimensions are the keys which index the documents that have non zero values for that dimension. Then for those docs which have non zero dimensions, the scores can be summed and computed. The highest scores will correspond to the most relevant documents.

This is one effective way of doing retrieval using a learning to rank model.